# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments           Customer  \
0  Mar  8 2023  8:44AM  258918        12    HH-40474          Hush Hush   
1  Mar  8 2023  8:44AM  258918        12    HH-40478          Hush Hush   
2  Mar  8 2023  8:44AM  258918        12    HH-40479          Hush Hush   
3  Mar  8 2023  8:44AM  258918        12    HH-40481          Hush Hush   
4  Mar  8 2023  8:43AM  258917        12    HH-40473          Hush Hush   
5  Mar  8 2023  8:39AM  258916        19     9242738     Zymergen, Inc.   
6  Mar  8 2023  8:35AM  258915        12      HH9098          Hush Hush   
7  Mar  8 2023  8:34AM  258914        12      HH9092          Hush Hush   
8  Mar  8 2023  8:34AM  258913        12      HH9074          Hush Hush   
9  Mar  8 2023  8:17AM  258912        22  03082022.1  NBTY Global, Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9      Executing

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
15  258914       Released          1
16  258915       Released          1
17  258916       Released          1
18  258917       Released          1
19  258918       Released          4

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
258914                NaN        NaN       1.0
258915                NaN        NaN       1.0
258916                NaN        NaN       1.0
258917                NaN        NaN       1.0
258918                NaN        NaN       4.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
258811               12.0        0.0       3.0
258816                0.0        1.0       0.0
258828                0.0        0.0       1.0
258862                0.0        0.0       1.0
258886                0.0        1.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
258811                 12          0         3
258816                  0          1         0
258828                  0          0         1
258862                  0          0         1
258886                  0          1         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               258811         12          0         3
1               258816          0          1         0
2               258828          0          0         1
3               258862          0          0         1
4               258886          0          1         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               258811        12                  3
1               258816                   1         
2               258828                            1
3               258862                            1
4               258886                   1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse             Customer
0   Mar  8 2023  8:44AM  258918        12            Hush Hush
4   Mar  8 2023  8:43AM  258917        12            Hush Hush
5   Mar  8 2023  8:39AM  258916        19       Zymergen, Inc.
6   Mar  8 2023  8:35AM  258915        12            Hush Hush
7   Mar  8 2023  8:34AM  258914        12            Hush Hush
8   Mar  8 2023  8:34AM  258913        12            Hush Hush
9   Mar  8 2023  8:17AM  258912        22    NBTY Global, Inc.
10  Mar  8 2023  8:07AM  258911        19     SD Head USA, LLC
12  Mar  8 2023  7:59AM  258910        22    NBTY Global, Inc.
13  Mar  8 2023  7:43AM  258906        19  AdvaGen Pharma, Ltd

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse             Customer Completed  \
0  Mar  8 2023  8:44AM  258918        12            Hush Hush             
1  Mar  8 2023  8:43AM  258917        12            Hush Hush             
2  Mar  8 2023  8:39AM  258916        19       Zymergen, Inc.             
3  Mar  8 2023  8:35AM  258915        12            Hush Hush             
4  Mar  8 2023  8:34AM  258914        12            Hush Hush             
5  Mar  8 2023  8:34AM  258913        12            Hush Hush             
6  Mar  8 2023  8:17AM  258912        22    NBTY Global, Inc.             
7  Mar  8 2023  8:07AM  258911        19     SD Head USA, LLC             
8  Mar  8 2023  7:59AM  258910        22    NBTY Global, Inc.             
9  Mar  8 2023  7:43AM  258906        19  AdvaGen Pharma, Ltd             

  Executing Released  
0                  4  
1                  1  
2                  1  
3                  1  
4                  1  
5                  1  
6         1           
7                  2  
8         1           
9                 23

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse             Customer Released  \
0  Mar  8 2023  8:44AM  258918        12            Hush Hush        4   
1  Mar  8 2023  8:43AM  258917        12            Hush Hush        1   
2  Mar  8 2023  8:39AM  258916        19       Zymergen, Inc.        1   
3  Mar  8 2023  8:35AM  258915        12            Hush Hush        1   
4  Mar  8 2023  8:34AM  258914        12            Hush Hush        1   
5  Mar  8 2023  8:34AM  258913        12            Hush Hush        1   
6  Mar  8 2023  8:17AM  258912        22    NBTY Global, Inc.            
7  Mar  8 2023  8:07AM  258911        19     SD Head USA, LLC        2   
8  Mar  8 2023  7:59AM  258910        22    NBTY Global, Inc.            
9  Mar  8 2023  7:43AM  258906        19  AdvaGen Pharma, Ltd       23   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6         1            
7                      
8         1            
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse        Customer Released Executing  \
0  Mar  8 2023  8:44AM  258918        12       Hush Hush        4             
1  Mar  8 2023  8:43AM  258917        12       Hush Hush        1             
2  Mar  8 2023  8:39AM  258916        19  Zymergen, Inc.        1             
3  Mar  8 2023  8:35AM  258915        12       Hush Hush        1             
4  Mar  8 2023  8:34AM  258914        12       Hush Hush        1             

  Completed  
0            
1            
2            
3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse        Customer  Released  Executing  \
0  Mar  8 2023  8:44AM  258918        12       Hush Hush       4.0        NaN   
1  Mar  8 2023  8:43AM  258917        12       Hush Hush       1.0        NaN   
2  Mar  8 2023  8:39AM  258916        19  Zymergen, Inc.       1.0        NaN   
3  Mar  8 2023  8:35AM  258915        12       Hush Hush       1.0        NaN   
4  Mar  8 2023  8:34AM  258914        12       Hush Hush       1.0        NaN   

   Completed  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse        Customer  Released  Executing  \
0  Mar  8 2023  8:44AM  258918        12       Hush Hush       4.0        0.0   
1  Mar  8 2023  8:43AM  258917        12       Hush Hush       1.0        0.0   
2  Mar  8 2023  8:39AM  258916        19  Zymergen, Inc.       1.0        0.0   
3  Mar  8 2023  8:35AM  258915        12       Hush Hush       1.0        0.0   
4  Mar  8 2023  8:34AM  258914        12       Hush Hush       1.0        0.0   

   Completed  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10          517673       4.0        0.0       12.0
12         2330079      10.0        2.0        0.0
19         1035561      27.0        0.0        0.0
21          517809       2.0        0.0        0.0
22          517822       0.0        2.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10   517673       4.0        0.0       12.0
1        12  2330079      10.0        2.0        0.0
2        19  1035561      27.0        0.0        0.0
3        21   517809       2.0        0.0        0.0
4        22   517822       0.0        2.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10       4.0        0.0       12.0
1        12      10.0        2.0        0.0
2        19      27.0        0.0        0.0
3        21       2.0        0.0        0.0
4        22       0.0        2.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released    4.0
1        12  Released   10.0
2        19  Released   27.0
3        21  Released    2.0
4        22  Released    0.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    12    19   21   22
Status                               
Completed  12.0   0.0   0.0  0.0  0.0
Executing   0.0   2.0   0.0  0.0  2.0
Released    4.0  10.0  27.0  2.0  0.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    12    19   21   22
0          Completed  12.0   0.0   0.0  0.0  0.0
1          Executing   0.0   2.0   0.0  0.0  2.0
2           Released   4.0  10.0  27.0  2.0  0.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    12    19   21   22
0  Completed  12.0   0.0   0.0  0.0  0.0
1  Executing   0.0   2.0   0.0  0.0  2.0
2   Released   4.0  10.0  27.0  2.0  0.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()